In [1]:
# Import dependencies
import pandas as pd
import getpass
from sqlalchemy import create_engine
# Visualizations
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as pxfrom
from plotly.subplots import make_subplots
import plotly.express as px
# Statisitcal analysis
from sklearn.metrics import r2_score
import statsmodels.api as sm
from scipy import stats




# Call DataFrame

In [2]:
# Enter password for connection string
password = getpass.getpass()

In [3]:
# Create connection string
conn_string = "postgresql://postgres:" + password + "@b-team-final-project.cct7ahzel1ur.us-west-2.rds.amazonaws.com:5432/B_Team_Final_Project"

In [4]:
# Create the connection
conn=create_engine(conn_string)

/Users/halesoyster/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [5]:
# Import locations_df
locations_df = pd.read_sql(
    'locations',
    conn)


In [6]:
locations_df['lat'] = locations_df['lat'].astype(float)
locations_df['lon'] = locations_df['lon'].astype(float)

In [7]:
locations_df.head(3)

,stateabbr,countyname,countyfips,lat,lon
0,NJ,Warren,34041,40.857163,-74.996717
1,OK,Jackson,40065,34.588118,-99.414674
2,IL,Lawrence,17101,38.720339,-87.726843


In [8]:
# Import health_outcomes table and drop "countyfips"
health_outcomes = pd.read_sql(
    'health_outcomes',
    conn
)
health_outcomes = health_outcomes.apply(pd.to_numeric)
health_outcomes = health_outcomes.drop("countyfips", axis=1)
health_outcomes.head(3)

,arthritis,casthma,bphigh,cancer,highchol,kidney,copd,chd,depression,diabetes,obesity,teethlost,stroke
0,23.0,9.7,34.1,6.1,31.3,3.1,7.1,6.3,21.2,11.8,36.3,18.8,3.3
1,25.6,9.5,33.3,6.5,29.3,2.9,8.0,6.4,21.2,10.7,38.8,17.2,3.4
2,23.7,9.7,32.9,6.4,30.4,2.9,6.7,5.8,19.0,11.0,37.0,17.2,3.1


In [9]:
# Import prevention table and drop "countyfips"
prev_df =  pd.read_sql(
    'prevention',
    conn
)
prev_df = prev_df.apply(pd.to_numeric)
prev_df = prev_df.drop("countyfips", axis=1)
prev_df.head(3)

,access,checkup,dental,bpmed,cholscreen,mammouse,cervical,colon_screen,corem,corew
0,24.2,74.1,61.1,57.9,83.8,71.5,83.0,58.7,25.2,23.7
1,18.9,73.4,57.8,56.8,83.0,68.7,82.2,61.2,24.7,20.6
2,20.5,72.6,59.3,56.8,82.1,69.0,82.6,58.4,24.4,22.3


In [10]:
# Import health_risk_behaviors table and drop "countyfips"
risk_df = pd.read_sql(
    'cancer_health_risk_merged',
    conn
)
risk_df = risk_df.apply(pd.to_numeric)
risk_df = risk_df.drop(columns=["countyfips", "cancer"], axis=1)
risk_df.head(3)

,density,total_population,binge,csmoking,lpa,sleep
0,94.3,56059,15.0,19.7,32.2,38.4
1,141.8,225463,19.0,19.9,27.6,36.4
2,27.8,24575,13.7,27.8,41.6,40.9


In [11]:
# Import Cancer Health Risks Data Frame table and drop "countyfips"
cancer_hr_df = pd.read_sql(
    'cancer_health_risk_merged',
    conn)
cancer_hr_df = cancer_hr_df.apply(pd.to_numeric)
cancer_hr_df = cancer_hr_df.drop('countyfips', axis=1)

cancer_hr_df.head()

,density,total_population,cancer,binge,csmoking,lpa,sleep
0,94.3,56059,6.5,15.0,19.7,32.2,38.4
1,141.8,225463,6.7,19.0,19.9,27.6,36.4
2,27.8,24575,6.2,13.7,27.8,41.6,40.9
3,36.0,22427,6.5,15.6,25.2,37.7,40.1
4,89.9,57997,6.7,15.7,22.2,33.3,38.8


In [12]:
cancer_hr_df['cancer'].describe()

count    3121.000000
mean        6.451458
std         0.363219
min         4.800000
25%         6.300000
50%         6.600000
75%         6.700000
max         7.300000
Name: cancer, dtype: float64

In [13]:
# different catagories for cancer and label as risk
cancer_risk = []
for risk in cancer_hr_df['cancer']:
    if risk <= 4.8 and risk >5:
        cancer_risk.append("4.5-5")
    elif risk >= 5 and risk < 5.5:
        cancer_risk.append("5-5.5")
    elif risk >= 5.5 and risk < 6:
        cancer_risk.append("5.5-6")
    elif risk >= 6 and risk < 6.5:
        cancer_risk.append("6-6.5")
    elif risk >= 6.5 and risk < 7:
        cancer_risk.append("6.5-7")
    else:
        cancer_risk.append("7+")

cancer_hr_df['cancer_risk'] = cancer_risk
cancer_hr_df['cancer_risk'].value_counts()

6.5-7    1931
6-6.5     868
5.5-6     199
5-5.5      73
7+         50
Name: cancer_risk, dtype: int64

In [14]:
cancer_hr_df.head(3)

,density,total_population,cancer,binge,csmoking,lpa,sleep,cancer_risk
0,94.3,56059,6.5,15.0,19.7,32.2,38.4,6.5-7
1,141.8,225463,6.7,19.0,19.9,27.6,36.4,6.5-7
2,27.8,24575,6.2,13.7,27.8,41.6,40.9,6-6.5


In [15]:
# Join the two DataFrames on Index
risk_location = cancer_hr_df.merge(locations_df, left_index=True, right_index=True)
risk_location.head(3)

,density,total_population,cancer,binge,csmoking,lpa,sleep,cancer_risk,stateabbr,countyname,countyfips,lat,lon
0,94.3,56059,6.5,15.0,19.7,32.2,38.4,6.5-7,NJ,Warren,34041,40.857163,-74.996717
1,141.8,225463,6.7,19.0,19.9,27.6,36.4,6.5-7,OK,Jackson,40065,34.588118,-99.414674
2,27.8,24575,6.2,13.7,27.8,41.6,40.9,6-6.5,IL,Lawrence,17101,38.720339,-87.726843


# Mapping county points

In [16]:
fig = px.scatter_mapbox(risk_location, lat="lat", lon="lon", text = 'countyname', zoom =3, size = 'total_population',
                        color = 'cancer', color_continuous_scale=px.colors.sequential.Viridis,
                        mapbox_style='open-street-map', title = 'County Populations and Average Rates of Cancer')
fig.show()

In [17]:
# Changing cancer_risk to float to visualize
#risk_location['cancer_risk'] = risk_location['cancer_risk'].astype(str)

In [18]:
#fig = px.scatter_mapbox(risk_location, lat="lat", lon="lon", text = 'countyname', zoom =3, size = 'total_population',
                        #color = 'cancer_risk', color_continuous_scale=px.colors.sequential.Viridis, mapbox_style='open-street-map')
#fig.show()

# Data Vizualization Sandbox

In [19]:
# join risk DFs to visualize 'access' and cancer column'
cancer_prev = cancer_hr_df.merge(prev_df, left_index=True, right_index=True)
cancer_prev.head(3)

,density,total_population,cancer,binge,csmoking,lpa,sleep,cancer_risk,access,checkup,dental,bpmed,cholscreen,mammouse,cervical,colon_screen,corem,corew
0,94.3,56059,6.5,15.0,19.7,32.2,38.4,6.5-7,24.2,74.1,61.1,57.9,83.8,71.5,83.0,58.7,25.2,23.7
1,141.8,225463,6.7,19.0,19.9,27.6,36.4,6.5-7,18.9,73.4,57.8,56.8,83.0,68.7,82.2,61.2,24.7,20.6
2,27.8,24575,6.2,13.7,27.8,41.6,40.9,6-6.5,20.5,72.6,59.3,56.8,82.1,69.0,82.6,58.4,24.4,22.3


In [20]:
fig = px.scatter(cancer_prev, x = 'total_population', y = 'access',
	             size='cancer', color= "cancer", color_continuous_scale=px.colors.sequential.Viridis,
                 log_x=True, title = 'Access to Health Insurance by Total Population and Cancer Rates')
fig.show()

In [21]:
fig = px.bar(cancer_prev, x = 'total_population', y = 'cancer_risk', color = 'access',
            orientation = 'h', hover_data =['access'], barmode = 'group', 
            title = 'Cancer Rates Grouped by County, Total Population and Health Insurance Access')

fig.show()

In [22]:


fig = go.Figure(go.Bar(
            x = cancer_prev['total_population'],
            y = cancer_prev['cancer_risk'],
            orientation='h',))

fig.show()

In [23]:


# y = cancer_prev['cancer_risk']
# y_access = cancer_prev['access']
# x = cancer_prev['cancer']


# # Creating two subplots
# fig = make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=True,
#                     shared_yaxes=False, vertical_spacing=0.001)

# fig.append_trace(go.Bar(
#     x=y,
#     y=x,
#     marker=dict(
#         color='rgba(50, 171, 96, 0.6)',
#         line=dict(
#             color='rgba(50, 171, 96, 1.0)',
#             width=1),
#     ),
#     name=' ',
#     orientation='h',
# ), 1, 1)

# fig.append_trace(go.Scatter(
#     x=y_access, y=x,
#     mode='lines+markers',
#     line_color='rgb(128, 0, 128)',
#     name=' ',
# ), 1, 2)

# fig.update_layout(
#     title=' ',
#     yaxis=dict(
#         showgrid=False,
#         showline=False,
#         showticklabels=True,
#         domain=[0, 0.85],
#     ),
#     yaxis2=dict(
#         showgrid=False,
#         showline=True,
#         showticklabels=False,
#         linecolor='rgba(102, 102, 102, 0.8)',
#         linewidth=2,
#         domain=[0, 0.85],
#     ),
#     xaxis=dict(
#         zeroline=False,
#         showline=False,
#         showticklabels=True,
#         showgrid=True,
#         domain=[0, 0.42],
#     ),
#     xaxis2=dict(
#         zeroline=False,
#         showline=False,
#         showticklabels=True,
#         showgrid=True,
#         domain=[0.47, 1],
#         side='top',
#         dtick=25000,
#     ),
#     legend=dict(x=0.029, y=1.038, font_size=10),
#     margin=dict(l=100, r=20, t=70, b=70),
#     paper_bgcolor='rgb(248, 248, 255)',
#     plot_bgcolor='rgb(248, 248, 255)',
# )


# fig.show()

In [26]:
cancer_hr_df.to_csv('data/cancer_hr_df.csv', index=False)

In [25]:
fig = go.Figure()

for column in cancer_hr_df.columns.to_list():
    fig.add_trace(
        go.Scatter(
            x = cancer_hr_df.index,
            y = cancer_hr_df[column],
            name = column
           
        )
    )
    
fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'All',
                  method = 'update',
                  args = [{'visible': [False, False, True, True, True, True, True, True]},
                          {'title': 'All',
                           'showlegend':True}]),
             dict(label = 'binge',
                  method = 'update',
                  args = [{'visible': [False, False, True, True, False, False, False, False]}, # the index of True aligns with the indices of plot traces
                          {'title': 'Binge Drinking',
                           'showlegend':True}]),
             dict(label = 'csmoking',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, True, False, False, False]},
                          {'title': 'Smoking',
                           'showlegend':True}]),
             dict(label = 'lpa',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, True, False, False]},
                          {'title': 'Limited Physical Activity',
                           'showlegend':True}]),
             dict(label = 'sleep',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, False, True]},
                          {'title': '> 7 Hours of Sleep',
                           'showlegend':True}]),
            
            ])
        )
    ])

fig.show()